In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [15]:
# Load dataset
df = pd.read_csv("synthetic_frustration_data.csv")

# Prepare input sequences
sequences = df["clicked_sequence"].apply(lambda x: list(map(int, str(x).split())))
max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_len, padding='post') / 9.0
X = X.reshape((X.shape[0], max_len, 1))

# Prepare output
y_raw = df["frustration_score"].values
scaler = MinMaxScaler()
y = scaler.fit_transform(y_raw.reshape(-1, 1)).flatten()

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Custom metrics
def tolerance_accuracy(y_true, y_pred):
    threshold = 0.1 / (K.max(y_true) - K.min(y_true))
    return K.mean(K.cast(K.abs(y_true - y_pred) <= threshold, dtype='float32'))

def regression_accuracy(y_true, y_pred):
    relative_error = K.abs(y_true - y_pred) / (K.abs(y_true) + K.epsilon())
    return K.mean(K.cast(relative_error <= 0.1, dtype='float32'))

# Build model
model = Sequential([
    Masking(mask_value=0.0, input_shape=(max_len, 1)),
    GRU(64, return_sequences=False),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae', tolerance_accuracy, regression_accuracy])

# Callbacks
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(patience=5, factor=0.5, verbose=1)
]

# Train model
model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

# Evaluate model
loss, mae, tol_acc, reg_acc = model.evaluate(X_test, y_test)
print(f"\nTest MAE (normalized): {mae:.4f}")
print(f"Tolerance Accuracy (±0.1 normalized): {tol_acc*100:.2f}%")
print(f"Regression Accuracy (≤10% relative error): {reg_acc*100:.2f}%")

# Predict and inverse transform
y_pred = model.predict(X_test)
y_pred_real = scaler.inverse_transform(y_pred)
y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1))

# Real-world MAE
real_mae = np.mean(np.abs(y_pred_real.flatten() - y_test_real.flatten()))
print(f"\nTest MAE (real scale): {real_mae:.4f}")

# Real-scale tolerance accuracy
def tolerance_accuracy_real(y_true, y_pred, tolerance):
    return np.mean(np.abs(y_true - y_pred) <= tolerance)

for tol in [0.1, 0.5, 1.0]:
    acc_real = tolerance_accuracy_real(y_test_real.flatten(), y_pred_real.flatten(), tolerance=tol)
    print(f"Tolerance Accuracy (±{tol} real scale): {acc_real*100:.2f}%")


Epoch 1/100
10/10 [==============================] - 5s 210ms/step - loss: 0.2908 - mae: 0.3890 - tolerance_accuracy: 0.2937 - regression_accuracy: 0.0688 - val_loss: 0.1905 - val_mae: 0.4033 - val_tolerance_accuracy: 0.0833 - val_regression_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 0s 10ms/step - loss: 0.1832 - mae: 0.3370 - tolerance_accuracy: 0.3063 - regression_accuracy: 0.0437 - val_loss: 0.1858 - val_mae: 0.3977 - val_tolerance_accuracy: 0.0833 - val_regression_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 0s 9ms/step - loss: 0.1117 - mae: 0.2581 - tolerance_accuracy: 0.4125 - regression_accuracy: 0.0938 - val_loss: 0.1511 - val_mae: 0.3531 - val_tolerance_accuracy: 0.2083 - val_regression_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 0s 9ms/step - loss: 0.1399 - mae: 0.2837 - tolerance_accuracy: 0.3812 - regression_accuracy: 0.0875 - val_loss: 0.1264 - v